# Eat Safe, Love


## Part 1: Database and Jupyter Notebook Set Up

In [1]:

# Import necessary libraries
from pymongo import MongoClient

# Connect to MongoDB on localhost and port 27017
client = MongoClient('mongodb://localhost:27017/')

# Connect to the 'uk_food' database (it will be created if it doesn't exist)
db = client['uk_food']

# Check the database connection by listing the collections
print("Collections in 'uk_food':", db.list_collection_names())

# Get the 'establishments' collection
establishments = db['establishments']


Collections in 'uk_food': []


## Part 2: Update the Database

1. Add the new restaurant "Penang Flavours".

In [ ]:

# Add a new restaurant "Penang Flavours" to the 'establishments' collection
new_restaurant = {
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": None,  # This will be updated later
    "LocalAuthorityName": "Greenwich",
    "Name": "Penang Flavours",
    "RatingValue": None,  # Not rated yet
    "Latitude": "51.4934",  # Example latitude
    "Longitude": "-0.0098"  
}

# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

# Check that the new restaurant was inserted
print("New restaurant inserted:", new_restaurant)


New restaurant inserted: {'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': None, 'LocalAuthorityName': 'Greenwich', 'Name': 'Penang Flavours', 'RatingValue': None, 'Latitude': '51.4934', 'Longitude': '-0.0098', '_id': ObjectId('67c67386fd66ce18393a34bc')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen".

In [3]:

# Find the BusinessTypeID for "Restaurant/Cafe/Canteen"
business_type = establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "BusinessType": 1})

# Display the BusinessTypeID for Restaurant/Cafe/Canteen
business_type


{'_id': ObjectId('67c67386fd66ce18393a34bc'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': None}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [5]:

# Update the new restaurant with the BusinessTypeID
establishments.update_one(
    {"Name": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type["BusinessTypeID"]}}
)

# Verify the update
updated_restaurant = establishments.find_one({"Name": "Penang Flavours"})
print("Updated Restaurant:", updated_restaurant)


Updated Restaurant: {'_id': ObjectId('67c67386fd66ce18393a34bc'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': None, 'LocalAuthorityName': 'Greenwich', 'Name': 'Penang Flavours', 'RatingValue': None, 'Latitude': '51.4934', 'Longitude': '-0.0098'}


4. Remove any establishments within the Dover Local Authority.

In [6]:

# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
dover_count


0

In [7]:

# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})

# Check if any remaining documents include Dover
dover_check = establishments.count_documents({"LocalAuthorityName": "Dover"})
dover_check


0

5. Convert `latitude`, `longitude`, and `RatingValue` to numeric types.

In [8]:

# Convert latitude and longitude fields to numeric values
establishments.update_many({}, {"$set": {
    "Latitude": {"$toDouble": "$Latitude"},
    "Longitude": {"$toDouble": "$Longitude"}
}})

# Set non-rated or invalid ratings to None and convert RatingValue to numeric
establishments.update_many(
    {"RatingValue": {"$in": ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]}},
    {"$set": {"RatingValue": None}}
)

establishments.update_many({}, {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}})

# Verify the updates
sample = establishments.find_one()
print("Sample document after conversion:", sample)


Sample document after conversion: {'_id': ObjectId('67c67386fd66ce18393a34bc'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': None, 'LocalAuthorityName': 'Greenwich', 'Name': 'Penang Flavours', 'RatingValue': {'$toInt': '$RatingValue'}, 'Latitude': {'$toDouble': '$Latitude'}, 'Longitude': {'$toDouble': '$Longitude'}}
